In [1]:
import duckdb
import pandas as pd
from src.model.utils import train_model
from src.preprocess.etl import get_dataframe
from src.model.inference import predict_month
from src.constants import PATH_DATABASE, MONTHS_BASELINE, MONTHS_INFERENCE, PARAMS


import warnings

warnings.filterwarnings("ignore")


In [2]:
df_temp = pd.read_csv("datasets/raw/data_for_test.csv")
df_temp.to_parquet("datasets/raw/data_for_test.parquet")

In [3]:
con = duckdb.connect(database=PATH_DATABASE, read_only=True)

In [4]:
con.sql(
    """
    SELECT
        numero_de_cliente,
        foto_mes,
        clase_ternaria_202109,
        LEAD(clase_ternaria_202109, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ASC) AS clase_ternaria_202108
    FROM competencia_03
    ORDER BY 
        numero_de_cliente,
        foto_mes
        
    """
).to_df().head(10000)

,numero_de_cliente,foto_mes,clase_ternaria_202109,ctarjeta_visa_transacciones_lag_1
0,4,202007,CONTINUA,CONTINUA
1,4,202008,CONTINUA,CONTINUA
2,4,202009,CONTINUA,CONTINUA
3,4,202010,CONTINUA,CONTINUA
4,4,202011,CONTINUA,CONTINUA
...,...,...,...,...
82,494389234,202105,CONTINUA,CONTINUA
83,494389234,202106,CONTINUA,CONTINUA
84,494389234,202107,CONTINUA,BAJA+1
85,494389234,202108,BAJA+1,BAJA+2


In [15]:
all_months = MONTHS_BASELINE + MONTHS_INFERENCE
all_months = [str(month) for month in all_months]
where_clause = ", ".join(all_months)

In [16]:
where_clause

'202007, 202008, 202009, 202010, 202011, 202012, 202101, 202102, 202103, 202104, 202105, 202106, 202107, 202108, 202109'

In [17]:
df = get_dataframe(con, where_clause)

In [18]:
df.head()

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,mautoservicio,ctarjeta_visa,ctarjeta_visa_transacciones,mtarjeta_visa_consumo,clase_ternaria_202104,clase_ternaria_202105,clase_ternaria_202106,clase_ternaria_202107,clase_ternaria_202108,clase_ternaria_202109
0,4231431,202007,0,0,1,51,47,2685.87,49834.16,2130.01,...,0.0,1,1,11810.23,CONTINUA,CONTINUA,CONTINUA,CONTINUA,CONTINUA,CONTINUA
1,4231431,202008,0,0,1,51,48,2264.82,45425.09,1981.17,...,0.0,1,1,12565.02,CONTINUA,CONTINUA,CONTINUA,CONTINUA,CONTINUA,CONTINUA
2,4231431,202009,0,0,1,51,49,2044.97,41781.22,1812.01,...,0.0,1,1,12564.98,CONTINUA,CONTINUA,CONTINUA,CONTINUA,CONTINUA,CONTINUA
3,4231431,202012,1,0,0,51,52,2522.09,35592.87,1857.05,...,0.0,1,1,12564.98,CONTINUA,CONTINUA,CONTINUA,CONTINUA,CONTINUA,CONTINUA
4,4231431,202101,1,0,0,52,53,3016.56,34162.13,2714.54,...,0.0,1,1,12564.98,CONTINUA,CONTINUA,CONTINUA,CONTINUA,CONTINUA,CONTINUA


In [19]:
drop_cols = [f"clase_ternaria_{i}" for i in MONTHS_INFERENCE]
drop_cols

['clase_ternaria_202104',
 'clase_ternaria_202105',
 'clase_ternaria_202106',
 'clase_ternaria_202107',
 'clase_ternaria_202108',
 'clase_ternaria_202109']

In [26]:
ground_truth = df[["numero_de_cliente","foto_mes", "clase_ternaria_202109"]]
ground_truth

,numero_de_cliente,foto_mes,clase_ternaria_202109
0,4231431,202007,CONTINUA
1,4231431,202008,CONTINUA
2,4231431,202009,CONTINUA
3,4231431,202012,CONTINUA
4,4231431,202101,CONTINUA
...,...,...,...
77,7,202109,BAJA+2
78,5,202007,CONTINUA
79,494389234,202007,CONTINUA
80,494389234,202012,CONTINUA


In [30]:
for month in MONTHS_INFERENCE:
    df["clase_binaria"] = df[f"clase_ternaria_{month}"].map({"BAJA+2":1, "BAJA+1":1, "CONTINUA":0})
    
    df_test = df[df["foto_mes"].isin([month])].copy()
    df_train = df[df["foto_mes"] <= int(month - 2)].copy()
    
    df_test = df_test.reset_index(drop=True)
    df_train = df_train.reset_index(drop=True)
    
    X_train = df_train.drop(columns=drop_cols, axis=1).copy()
    X_test = df_test.drop(columns=drop_cols, axis=1).copy()
    
    y_train = df_train["clase_binaria"].copy()
    
    tags = {
        'stage': 'monthly',
        'last_month': month,
    }
    
    model = train_model(X_train, y_train, PARAMS, tags)
    
    preds = X_test[["numero_de_cliente", "foto_mes"]].copy()
    preds["prediction"] = predict_month(model, X_test)
    preds = preds.merge(ground_truth, on=["numero_de_cliente", "foto_mes"])
    
    preds.to_csv(f"datasets/processed/predictions/preds_{month}.csv", index=False)

2023/11/13 21:51:44 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during lightgbm autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 6fb7a3b173d74111b7ec9edde4b675a1. Failed operations: [MlflowException("Duplicate parameter keys have been submitted: [\'num_boost_round\']. Please ensure the request contains only one param value per param key.")]')]
2023/11/13 21:51:45 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during lightgbm autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID a42b2fff32b74461ba40692f995faf7c. Failed operations: [MlflowException("Duplicate parameter keys have been submitted: [\'num_boost_round\']. Please ensure the request contains only one param value per param key.")]')]
2023/11/13 2